In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold,train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
 
from scipy.stats import skew
from collections import OrderedDict

In [2]:
train= pd.read_csv('/content/drive/MyDrive/train.csv')
train.head()

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,cmc_0_0_0_1000,cmc_0_0_0_2_grad,cmc_0_0_0_2_interpolated,cmc_0_0_0_2_next,cmc_0_0_0_2,cmc_0_0_0_500,cmc_0_0_0_700,cmc_0_0_0_850,cmc_0_0_0_925,cmc_0_0_6_2,cmc_0_0_7_1000,cmc_0_0_7_2,cmc_0_0_7_500,cmc_0_0_7_700,cmc_0_0_7_850,cmc_0_0_7_925,cmc_0_1_0_0,cmc_0_1_11_0,cmc_0_1_65_0,cmc_0_1_66_0,cmc_0_1_67_0,cmc_0_1_68_0,cmc_0_1_7_0,cmc_0_2_2_10,cmc_0_2_2_1000,cmc_0_2_2_500,cmc_0_2_2_700,cmc_0_2_2_850,cmc_0_2_2_925,cmc_0_2_3_10,...,gfs_temperature_90000,gfs_temperature_92500,gfs_temperature_95000,gfs_temperature_97500,gfs_temperature_sea,gfs_temperature_sea_grad,gfs_temperature_sea_interpolated,gfs_temperature_sea_next,gfs_timedelta_s,gfs_total_clouds_cover_high,gfs_total_clouds_cover_low,gfs_total_clouds_cover_middle,gfs_u_wind,gfs_v_wind,gfs_wind_speed,wrf_available,wrf_t2,wrf_t2_next,wrf_psfc,wrf_rh2,wrf_wind_u,wrf_wind_v,wrf_rain,wrf_snow,wrf_graupel,wrf_hail,wrf_t2_interpolated,wrf_t2_grad,cmc_0_1_65_0_grad,cmc_0_1_65_0_next,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,127.0,-17.526443,14.613571,754.263405,282.702881,6.684790,276.300439,282.985229,276.300439,262.333789,276.962134,280.083057,281.141064,272.500946,14.250,3.750,29.905515,15.500,29.750,15.750,0.0034,0.0,0.00000,0.0,0.0,0.0,0.00000,-1.327548,-2.914091,18.357608,8.480661,-1.510544,-0.465033,-0.126199,...,7.345331,7.926080,8.841516,9.350854,9.077112,1.966156,9.077112,11.043268,0.0,0.0,0.0,0.0,-2.328149,0.509014,2.383144,1.0,280.888000,280.420593,100927.414062,0.623658,-2.533346,-3.091468,0.0,0.0,0.0,0.0,280.888000,-0.467407,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-2.600006,-2.750006,0.0,0.0
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,297.0,41.531032,26.992143,733.117168,305.860669,-1.361548,303.629541,302.267993,303.629541,267.178882,286.211719,291.642969,298.615479,293.701019,11.500,9.875,22.468424,26.500,1.375,6.250,0.0172,0.0,5.67340,0.0,0.0,0.0,0.00000,-3.568011,-3.632005,4.370947,0.653320,-3.227249,-4.499048,7.287793,...,24.699792,27.050012,29.526605,31.950006,32.831812,-2.257446,32.831812,30.574365,0.0,0.0,23.0,0.0,-3.851841,7.031523,8.017418,1.0,302.893005,303.356781,97645.507812,0.651412,-3.482976,5.238701,0.0,0.0,0.0,0.0,302.893005,0.463776,0.000000,5.673400,0.0,0.0,0.0,0.0,0.0,0.0,-0.600006,17.950006,-12.0,11.0
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.0,43.916531,18.842143,761.571076,294.469702,0.282837,295.153361,295.341919,295.059082,268.584375,280.319653,288.922583,292.259375,293.914505,0.750,1.000,25.403430,1.625,2.375,0.125,0.0161,0.0,82.66500,0.0,0.0,0.0,0.00003,0.674461,0.812148,28.046518,20.135425,11.603281,5.047708,1.483844,...,17.325555,18.692010,20.252496,21.868280,23.101434,0.287476,23.197260,23.388910,3600.0,0.0,7.0,0.0,1.512065,4.074417,4.345942,1.0,295.782043,295.726929,100751.773438,0.989764,3.433286,5.181113,0.0,0.0,0.0,0.0,295.782043,-0.055115,0.771400,83.436400,0.0,0.0,0.0,0.0,0.0,0.0,-0.233978,21.450006,1.0,8.0
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,193.0,40.240955,8.303571,747.524910,295.118774,1.091162,296.376611,297.467773,296.376611,262.566675,278.622852,285.923389,290.274927,288.752838,6.125,7.500,5.767445,14.000,4.875,4.125,0.0120,0.0,4.08775,0.0,0.0,0.0,0.00000,2.791129,2.760229,29.527142,25.678796,14.091809,6.503445,6.488293,...,12.791437,13.875513,15.751978,17.743524,19.321313,2.133026,19.321313,21.454340,0.0,21.0,99.0,0.0,2.277510,7.151445,7.505346,1.0,295.235687,296.000824,99450.273438,0.749565,-0.158933,8.494842,0.0,0.0,0.0,0.0,295.235687,0.765137,0.000000,4.087750,0.0,0.0,0.0,0.0,0.0,0.0,0.059448,16.150018,-58.0,41.0
4,1.552910e+09,46.066667,41.966667,9.0,20.0,dry,90.0,30.394660,6.451429,753.168113,28

In [3]:
dev_in= pd.read_csv('/content/drive/MyDrive/dev_in.csv')
dev_in

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,cmc_0_0_0_1000,cmc_0_0_0_2_grad,cmc_0_0_0_2_interpolated,cmc_0_0_0_2_next,cmc_0_0_0_2,cmc_0_0_0_500,cmc_0_0_0_700,cmc_0_0_0_850,cmc_0_0_0_925,cmc_0_0_6_2,cmc_0_0_7_1000,cmc_0_0_7_2,cmc_0_0_7_500,cmc_0_0_7_700,cmc_0_0_7_850,cmc_0_0_7_925,cmc_0_1_0_0,cmc_0_1_11_0,cmc_0_1_65_0,cmc_0_1_66_0,cmc_0_1_67_0,cmc_0_1_68_0,cmc_0_1_7_0,cmc_0_2_2_10,cmc_0_2_2_1000,cmc_0_2_2_500,cmc_0_2_2_700,cmc_0_2_2_850,cmc_0_2_2_925,cmc_0_2_3_10,...,gfs_temperature_90000,gfs_temperature_92500,gfs_temperature_95000,gfs_temperature_97500,gfs_temperature_sea,gfs_temperature_sea_grad,gfs_temperature_sea_interpolated,gfs_temperature_sea_next,gfs_timedelta_s,gfs_total_clouds_cover_high,gfs_total_clouds_cover_low,gfs_total_clouds_cover_middle,gfs_u_wind,gfs_v_wind,gfs_wind_speed,wrf_available,wrf_t2,wrf_t2_next,wrf_psfc,wrf_rh2,wrf_wind_u,wrf_wind_v,wrf_rain,wrf_snow,wrf_graupel,wrf_hail,wrf_t2_interpolated,wrf_t2_grad,cmc_0_1_65_0_grad,cmc_0_1_65_0_next,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,282.619775,0.426294,284.084521,284.510815,284.084521,254.674902,268.638867,274.181128,277.269458,280.656006,3.250,3.250,8.884214,23.375,5.875,0.125,0.007800,0.00,0.710750,0.000000,0.000000,0.0,0.000000,6.366026,7.387439,20.139139,18.186014,11.808014,9.107840,-0.145898,...,-1.750006,0.249994,2.303857,4.334222,8.346277,0.444611,8.346277,8.790887,0.0,0.0,0.0,0.0,2.485703,6.422219,6.886481,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007600,0.718350,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,279.546143,-0.734399,277.743530,277.009131,277.743530,243.591406,261.825244,272.102686,277.477319,276.712234,2.375,1.000,9.000224,7.000,1.000,2.500,0.005400,0.00,17.608325,0.000000,0.000000,0.0,0.000000,3.452992,3.252415,17.576198,12.145520,15.007712,14.071326,2.931961,...,1.756708,3.153925,3.989069,3.411646,2.690546,0.185333,2.690546,2.875879,0.0,56.0,0.0,30.0,2.282283,3.202654,3.932659,1,276.833740,276.415131,100473.835938,0.935304,5.423971,1.841448,0.000000,0.000000,0.000000,0.0,276.833740,-0.418610,0.000075,17.608400,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,295.376318,0.137329,296.955811,297.047363,296.910034,265.435693,280.136548,288.021509,290.656055,293.811935,2.250,2.875,19.526672,5.250,2.875,0.375,0.018300,0.00,0.502250,0.000000,0.000000,0.0,0.000000,-4.727408,-5.139154,10.135144,3.114999,-0.702115,-3.545020,-3.017508,...,17.500208,18.850000,18.842828,19.849969,23.527246,0.195007,23.592249,23.722253,3600.0,100.0,2.0,0.0,-4.176856,-2.449009,4.841876,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.502250,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,284.379980,-0.461279,273.738379,273.430859,273.892139,251.706055,267.990430,276.035034,280.517212,273.591858,7.750,0.125,4.492367,0.750,1.625,4.875,0.004900,0.20,12.078150,3.468580,0.000049,0.0,0.000195,-0.054332,-0.043832,0.821658,1.102182,-0.047452,-0.076706,0.942397,...,8.149988,9.649988,11.150018,12.568719,2.416406,-1.114319,2.044967,1.302087,3600.0,78.0,93.0,100.0,-1.210991,4.430346,4.592872,1,278.312378,277.809601,84022.960938,0.884510,1.734115,1.362197,0.949750,0.000075,0.000028,0.0,278.312378,-0.502777,0.059500,12.137650,1.69672,5.165300,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.6

In [4]:
dev_out= pd.read_csv('/content/drive/MyDrive/dev_out.csv')
dev_out

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,cmc_0_0_0_1000,cmc_0_0_0_2_grad,cmc_0_0_0_2_interpolated,cmc_0_0_0_2_next,cmc_0_0_0_2,cmc_0_0_0_500,cmc_0_0_0_700,cmc_0_0_0_850,cmc_0_0_0_925,cmc_0_0_6_2,cmc_0_0_7_1000,cmc_0_0_7_2,cmc_0_0_7_500,cmc_0_0_7_700,cmc_0_0_7_850,cmc_0_0_7_925,cmc_0_1_0_0,cmc_0_1_11_0,cmc_0_1_65_0,cmc_0_1_66_0,cmc_0_1_67_0,cmc_0_1_68_0,cmc_0_1_7_0,cmc_0_2_2_10,cmc_0_2_2_1000,cmc_0_2_2_500,cmc_0_2_2_700,cmc_0_2_2_850,cmc_0_2_2_925,cmc_0_2_3_10,...,gfs_temperature_90000,gfs_temperature_92500,gfs_temperature_95000,gfs_temperature_97500,gfs_temperature_sea,gfs_temperature_sea_grad,gfs_temperature_sea_interpolated,gfs_temperature_sea_next,gfs_timedelta_s,gfs_total_clouds_cover_high,gfs_total_clouds_cover_low,gfs_total_clouds_cover_middle,gfs_u_wind,gfs_v_wind,gfs_wind_speed,wrf_available,wrf_t2,wrf_t2_next,wrf_psfc,wrf_rh2,wrf_wind_u,wrf_wind_v,wrf_rain,wrf_snow,wrf_graupel,wrf_hail,wrf_t2_interpolated,wrf_t2_grad,cmc_0_1_65_0_grad,cmc_0_1_65_0_next,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1563124500,42.614899,-89.590401,28.0,0,snow,313.0,65.438862,19.171429,737.750555,298.114404,3.482544,300.603483,301.764331,298.281787,264.616016,283.003857,290.596753,294.996313,293.024185,5.000,5.125,10.500000,25.375,12.500,10.250,0.0170,0.0,5.519825,0.0,0.0,0.0,0.0,-1.636842,-1.605179,6.745984,4.536145,1.651538,-1.466791,-0.692841,...,19.150018,20.350000,19.749994,20.950006,22.778101,3.171875,24.892684,25.949976,7200.0,0.0,45.0,0.0,-1.608711,2.780185,3.212068,1,298.230286,299.220520,98325.906250,1.000000,-0.868262,1.657977,0.300395,0.0,0.0,0.0,298.230286,0.990234,0.000025,5.519850,0.0,0.0,0.0,0.0,0.0,0.0,2.399994,23.450006,26.0,71.0
1,1565951400,61.885201,34.154701,18.0,10,snow,34.0,40.277528,15.127857,751.846067,289.922168,0.461267,291.023409,291.330920,290.869653,251.816846,271.103027,277.059863,283.241772,282.840561,8.250,8.000,9.875000,17.000,0.625,3.500,0.0090,0.0,0.182175,0.0,0.0,0.0,0.0,4.471985,5.292297,7.113849,8.157370,8.872421,7.860455,0.540353,...,8.150018,10.250024,12.406396,14.606409,17.649988,1.700012,18.216659,19.350000,3600.0,0.0,19.0,0.0,4.746643,1.079504,4.867849,1,288.700226,287.255615,101065.085938,0.818138,3.338219,1.081676,0.358113,0.0,0.0,0.0,288.700226,-1.444611,0.021250,0.203425,0.0,0.0,0.0,0.0,0.0,0.0,-0.300018,9.449976,21.0,40.0
2,1563850200,38.468201,43.332298,17.0,10,snow,1652.0,-0.237116,20.117857,609.203551,301.351074,-0.192773,291.672510,291.608252,291.801025,269.125879,289.468872,295.645630,298.293115,280.200348,16.375,11.500,18.625000,17.375,13.000,14.375,0.0171,0.0,0.159850,0.0,0.0,0.0,0.0,-1.284781,-1.301263,0.780786,-2.341394,-1.277808,-1.275647,2.451781,...,29.028223,30.601007,32.050043,33.850000,13.749994,2.399994,15.349990,16.149988,7200.0,0.0,0.0,0.0,0.109431,0.872329,0.879166,1,293.327301,293.896698,82966.734375,0.562927,-3.344606,3.610907,0.000000,0.0,0.0,0.0,293.327301,0.569397,0.000000,0.159850,0.0,0.0,0.0,0.0,0.0,0.0,1.849335,3.365320,0.0,0.0
3,1562806380,38.791000,-97.652199,28.0,0,snow,383.0,19.207866,33.193571,724.725255,306.146948,-4.270203,303.320825,299.050623,303.320825,266.845483,285.047485,293.598804,299.299219,292.477844,13.250,10.750,19.000000,14.750,5.625,8.875,0.0161,0.0,2.589825,0.0,0.0,0.0,0.0,-1.755324,-1.779726,9.914410,8.539859,-1.212570,-2.344110,-3.477444,...,21.950006,24.249994,26.550012,28.349969,27.749994,-4.914856,27.749994,22.835138,0.0,0.0,0.0,0.0,-2.274773,-4.191489,4.768980,1,303.776550,301.418518,97246.890625,0.414574,-1.316880,-3.560403,0.000000,0.0,0.0,0.0,303.776550,-2.358032,0.000000,2.589825,0.0,0.0,0.0,0.0,0.0,0.0,-2.486816,16.263177,0.0,0.0
4,1564509600,58.767200,-111.116997,20.0,20,snow,235.0,46.174496,19.481429,740.954837,294.056982,1.473193,294.807349,296.28054

In [5]:
train.fillna(0)

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,cmc_0_0_0_1000,cmc_0_0_0_2_grad,cmc_0_0_0_2_interpolated,cmc_0_0_0_2_next,cmc_0_0_0_2,cmc_0_0_0_500,cmc_0_0_0_700,cmc_0_0_0_850,cmc_0_0_0_925,cmc_0_0_6_2,cmc_0_0_7_1000,cmc_0_0_7_2,cmc_0_0_7_500,cmc_0_0_7_700,cmc_0_0_7_850,cmc_0_0_7_925,cmc_0_1_0_0,cmc_0_1_11_0,cmc_0_1_65_0,cmc_0_1_66_0,cmc_0_1_67_0,cmc_0_1_68_0,cmc_0_1_7_0,cmc_0_2_2_10,cmc_0_2_2_1000,cmc_0_2_2_500,cmc_0_2_2_700,cmc_0_2_2_850,cmc_0_2_2_925,cmc_0_2_3_10,...,gfs_temperature_90000,gfs_temperature_92500,gfs_temperature_95000,gfs_temperature_97500,gfs_temperature_sea,gfs_temperature_sea_grad,gfs_temperature_sea_interpolated,gfs_temperature_sea_next,gfs_timedelta_s,gfs_total_clouds_cover_high,gfs_total_clouds_cover_low,gfs_total_clouds_cover_middle,gfs_u_wind,gfs_v_wind,gfs_wind_speed,wrf_available,wrf_t2,wrf_t2_next,wrf_psfc,wrf_rh2,wrf_wind_u,wrf_wind_v,wrf_rain,wrf_snow,wrf_graupel,wrf_hail,wrf_t2_interpolated,wrf_t2_grad,cmc_0_1_65_0_grad,cmc_0_1_65_0_next,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,127.0,-17.526443,14.613571,754.263405,282.702881,6.684790,276.300439,282.985229,276.300439,262.333789,276.962134,280.083057,281.141064,272.500946,14.250,3.750,29.905515,15.500,29.750,15.750,0.0034,0.0,0.000000,0.00000,0.0,0.0,0.00000,-1.327548,-2.914091,18.357608,8.480661,-1.510544,-0.465033,-0.126199,...,7.345331,7.926080,8.841516,9.350854,9.077112,1.966156,9.077112,11.043268,0.0,0.0,0.0,0.0,-2.328149,0.509014,2.383144,1.0,280.888000,280.420593,100927.414062,0.623658,-2.533346,-3.091468,0.000000e+00,0.000000e+00,0.000000e+00,0.0,280.888000,-0.467407,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,-2.600006,-2.750006,0.0,0.0
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,297.0,41.531032,26.992143,733.117168,305.860669,-1.361548,303.629541,302.267993,303.629541,267.178882,286.211719,291.642969,298.615479,293.701019,11.500,9.875,22.468424,26.500,1.375,6.250,0.0172,0.0,5.673400,0.00000,0.0,0.0,0.00000,-3.568011,-3.632005,4.370947,0.653320,-3.227249,-4.499048,7.287793,...,24.699792,27.050012,29.526605,31.950006,32.831812,-2.257446,32.831812,30.574365,0.0,0.0,23.0,0.0,-3.851841,7.031523,8.017418,1.0,302.893005,303.356781,97645.507812,0.651412,-3.482976,5.238701,0.000000e+00,0.000000e+00,0.000000e+00,0.0,302.893005,0.463776,0.000000,5.673400,0.000000,0.000000,0.0,0.0,0.0,0.0,-0.600006,17.950006,-12.0,11.0
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.0,43.916531,18.842143,761.571076,294.469702,0.282837,295.153361,295.341919,295.059082,268.584375,280.319653,288.922583,292.259375,293.914505,0.750,1.000,25.403430,1.625,2.375,0.125,0.0161,0.0,82.665000,0.00000,0.0,0.0,0.00003,0.674461,0.812148,28.046518,20.135425,11.603281,5.047708,1.483844,...,17.325555,18.692010,20.252496,21.868280,23.101434,0.287476,23.197260,23.388910,3600.0,0.0,7.0,0.0,1.512065,4.074417,4.345942,1.0,295.782043,295.726929,100751.773438,0.989764,3.433286,5.181113,0.000000e+00,0.000000e+00,0.000000e+00,0.0,295.782043,-0.055115,0.771400,83.436400,0.000000,0.000000,0.0,0.0,0.0,0.0,-0.233978,21.450006,1.0,8.0
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,193.0,40.240955,8.303571,747.524910,295.118774,1.091162,296.376611,297.467773,296.376611,262.566675,278.622852,285.923389,290.274927,288.752838,6.125,7.500,5.767445,14.000,4.875,4.125,0.0120,0.0,4.087750,0.00000,0.0,0.0,0.00000,2.791129,2.760229,29.527142,25.678796,14.091809,6.503445,6.488293,...,12.791437,13.875513,15.751978,17.743524,19.321313,2.133026,19.321313,21.454340,0.0,21.0,99.0,0.0,2.277510,7.151445,7.505346,1.0,295.235687,296.000824,99450.273438,0.749565,-0.158933,8.494842,0.000000e+00,0.000000e+00,0.000000e+00,0.0,295.235687,0.765137

In [6]:
dev_in.fillna(0)

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,cmc_0_0_0_1000,cmc_0_0_0_2_grad,cmc_0_0_0_2_interpolated,cmc_0_0_0_2_next,cmc_0_0_0_2,cmc_0_0_0_500,cmc_0_0_0_700,cmc_0_0_0_850,cmc_0_0_0_925,cmc_0_0_6_2,cmc_0_0_7_1000,cmc_0_0_7_2,cmc_0_0_7_500,cmc_0_0_7_700,cmc_0_0_7_850,cmc_0_0_7_925,cmc_0_1_0_0,cmc_0_1_11_0,cmc_0_1_65_0,cmc_0_1_66_0,cmc_0_1_67_0,cmc_0_1_68_0,cmc_0_1_7_0,cmc_0_2_2_10,cmc_0_2_2_1000,cmc_0_2_2_500,cmc_0_2_2_700,cmc_0_2_2_850,cmc_0_2_2_925,cmc_0_2_3_10,...,gfs_temperature_90000,gfs_temperature_92500,gfs_temperature_95000,gfs_temperature_97500,gfs_temperature_sea,gfs_temperature_sea_grad,gfs_temperature_sea_interpolated,gfs_temperature_sea_next,gfs_timedelta_s,gfs_total_clouds_cover_high,gfs_total_clouds_cover_low,gfs_total_clouds_cover_middle,gfs_u_wind,gfs_v_wind,gfs_wind_speed,wrf_available,wrf_t2,wrf_t2_next,wrf_psfc,wrf_rh2,wrf_wind_u,wrf_wind_v,wrf_rain,wrf_snow,wrf_graupel,wrf_hail,wrf_t2_interpolated,wrf_t2_grad,cmc_0_1_65_0_grad,cmc_0_1_65_0_next,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,282.619775,0.426294,284.084521,284.510815,284.084521,254.674902,268.638867,274.181128,277.269458,280.656006,3.250,3.250,8.884214,23.375,5.875,0.125,0.007800,0.00,0.710750,0.000000,0.000000,0.0,0.000000,6.366026,7.387439,20.139139,18.186014,11.808014,9.107840,-0.145898,...,-1.750006,0.249994,2.303857,4.334222,8.346277,0.444611,8.346277,8.790887,0.0,0.0,0.0,0.0,2.485703,6.422219,6.886481,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.007600,0.718350,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,279.546143,-0.734399,277.743530,277.009131,277.743530,243.591406,261.825244,272.102686,277.477319,276.712234,2.375,1.000,9.000224,7.000,1.000,2.500,0.005400,0.00,17.608325,0.000000,0.000000,0.0,0.000000,3.452992,3.252415,17.576198,12.145520,15.007712,14.071326,2.931961,...,1.756708,3.153925,3.989069,3.411646,2.690546,0.185333,2.690546,2.875879,0.0,56.0,0.0,30.0,2.282283,3.202654,3.932659,1,276.833740,276.415131,100473.835938,0.935304,5.423971,1.841448,0.000000,0.000000,0.000000,0.0,276.833740,-0.418610,0.000075,17.608400,0.00000,0.000000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,295.376318,0.137329,296.955811,297.047363,296.910034,265.435693,280.136548,288.021509,290.656055,293.811935,2.250,2.875,19.526672,5.250,2.875,0.375,0.018300,0.00,0.502250,0.000000,0.000000,0.0,0.000000,-4.727408,-5.139154,10.135144,3.114999,-0.702115,-3.545020,-3.017508,...,17.500208,18.850000,18.842828,19.849969,23.527246,0.195007,23.592249,23.722253,3600.0,100.0,2.0,0.0,-4.176856,-2.449009,4.841876,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.502250,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,284.379980,-0.461279,273.738379,273.430859,273.892139,251.706055,267.990430,276.035034,280.517212,273.591858,7.750,0.125,4.492367,0.750,1.625,4.875,0.004900,0.20,12.078150,3.468580,0.000049,0.0,0.000195,-0.054332,-0.043832,0.821658,1.102182,-0.047452,-0.076706,0.942397,...,8.149988,9.649988,11.150018,12.568719,2.416406,-1.114319,2.044967,1.302087,3600.0,78.0,93.0,100.0,-1.210991,4.430346,4.592872,1,278.312378,277.809601,84022.960938,0.884510,1.734115,1.362197,0.949750,0.000075,0.000028,0.0,278.312378,-0.502777,0.059500,12.137650,1.69672,5.165300,0.0,0.000049,0.0,0.0,-1.500000

In [7]:
dev_out.fillna(0)

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,cmc_0_0_0_1000,cmc_0_0_0_2_grad,cmc_0_0_0_2_interpolated,cmc_0_0_0_2_next,cmc_0_0_0_2,cmc_0_0_0_500,cmc_0_0_0_700,cmc_0_0_0_850,cmc_0_0_0_925,cmc_0_0_6_2,cmc_0_0_7_1000,cmc_0_0_7_2,cmc_0_0_7_500,cmc_0_0_7_700,cmc_0_0_7_850,cmc_0_0_7_925,cmc_0_1_0_0,cmc_0_1_11_0,cmc_0_1_65_0,cmc_0_1_66_0,cmc_0_1_67_0,cmc_0_1_68_0,cmc_0_1_7_0,cmc_0_2_2_10,cmc_0_2_2_1000,cmc_0_2_2_500,cmc_0_2_2_700,cmc_0_2_2_850,cmc_0_2_2_925,cmc_0_2_3_10,...,gfs_temperature_90000,gfs_temperature_92500,gfs_temperature_95000,gfs_temperature_97500,gfs_temperature_sea,gfs_temperature_sea_grad,gfs_temperature_sea_interpolated,gfs_temperature_sea_next,gfs_timedelta_s,gfs_total_clouds_cover_high,gfs_total_clouds_cover_low,gfs_total_clouds_cover_middle,gfs_u_wind,gfs_v_wind,gfs_wind_speed,wrf_available,wrf_t2,wrf_t2_next,wrf_psfc,wrf_rh2,wrf_wind_u,wrf_wind_v,wrf_rain,wrf_snow,wrf_graupel,wrf_hail,wrf_t2_interpolated,wrf_t2_grad,cmc_0_1_65_0_grad,cmc_0_1_65_0_next,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1563124500,42.614899,-89.590401,28.0,0,snow,313.0,65.438862,19.171429,737.750555,298.114404,3.482544,300.603483,301.764331,298.281787,264.616016,283.003857,290.596753,294.996313,293.024185,5.000,5.125,10.500000,25.375,12.500,10.250,0.0170,0.0,5.519825,0.0,0.0,0.0,0.0,-1.636842,-1.605179,6.745984,4.536145,1.651538,-1.466791,-0.692841,...,19.150018,20.350000,19.749994,20.950006,22.778101,3.171875,24.892684,25.949976,7200.0,0.0,45.0,0.0,-1.608711,2.780185,3.212068,1,298.230286,299.220520,98325.906250,1.000000,-0.868262,1.657977,0.300395,0.0,0.0,0.0,298.230286,0.990234,0.000025,5.519850,0.0,0.0,0.0,0.0,0.0,0.0,2.399994,23.450006,26.0,71.0
1,1565951400,61.885201,34.154701,18.0,10,snow,34.0,40.277528,15.127857,751.846067,289.922168,0.461267,291.023409,291.330920,290.869653,251.816846,271.103027,277.059863,283.241772,282.840561,8.250,8.000,9.875000,17.000,0.625,3.500,0.0090,0.0,0.182175,0.0,0.0,0.0,0.0,4.471985,5.292297,7.113849,8.157370,8.872421,7.860455,0.540353,...,8.150018,10.250024,12.406396,14.606409,17.649988,1.700012,18.216659,19.350000,3600.0,0.0,19.0,0.0,4.746643,1.079504,4.867849,1,288.700226,287.255615,101065.085938,0.818138,3.338219,1.081676,0.358113,0.0,0.0,0.0,288.700226,-1.444611,0.021250,0.203425,0.0,0.0,0.0,0.0,0.0,0.0,-0.300018,9.449976,21.0,40.0
2,1563850200,38.468201,43.332298,17.0,10,snow,1652.0,-0.237116,20.117857,609.203551,301.351074,-0.192773,291.672510,291.608252,291.801025,269.125879,289.468872,295.645630,298.293115,280.200348,16.375,11.500,18.625000,17.375,13.000,14.375,0.0171,0.0,0.159850,0.0,0.0,0.0,0.0,-1.284781,-1.301263,0.780786,-2.341394,-1.277808,-1.275647,2.451781,...,29.028223,30.601007,32.050043,33.850000,13.749994,2.399994,15.349990,16.149988,7200.0,0.0,0.0,0.0,0.109431,0.872329,0.879166,1,293.327301,293.896698,82966.734375,0.562927,-3.344606,3.610907,0.000000,0.0,0.0,0.0,293.327301,0.569397,0.000000,0.159850,0.0,0.0,0.0,0.0,0.0,0.0,1.849335,3.365320,0.0,0.0
3,1562806380,38.791000,-97.652199,28.0,0,snow,383.0,19.207866,33.193571,724.725255,306.146948,-4.270203,303.320825,299.050623,303.320825,266.845483,285.047485,293.598804,299.299219,292.477844,13.250,10.750,19.000000,14.750,5.625,8.875,0.0161,0.0,2.589825,0.0,0.0,0.0,0.0,-1.755324,-1.779726,9.914410,8.539859,-1.212570,-2.344110,-3.477444,...,21.950006,24.249994,26.550012,28.349969,27.749994,-4.914856,27.749994,22.835138,0.0,0.0,0.0,0.0,-2.274773,-4.191489,4.768980,1,303.776550,301.418518,97246.890625,0.414574,-1.316880,-3.560403,0.000000,0.0,0.0,0.0,303.776550,-2.358032,0.000000,2.589825,0.0,0.0,0.0,0.0,0.0,0.0,-2.486816,16.263177,0.0,0.0
4,1564509600,58.767200,-111.116997,20.0,20,snow,235.0,46.174496,19.481429,740.954837,294.056982,1.473193,294.807349,296.28054

In [8]:
X_train = train.iloc[:,6:]
X_dev_in = dev_in.iloc[:,6:]
X_dev_out = dev_out.iloc[:,6:]
y_train = train['fact_temperature']
y_dev_in = dev_in['fact_temperature']
y_dev_out = dev_out['fact_temperature']

In [9]:
model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0, 
                 objective ='reg:squarederror',
                 learning_rate=0.3,
                 max_depth=6,
                 min_child_weight=1,
                 n_estimators=100,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [ ]:
model.fit(X_train,y_train)
sorted_=OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))
sorted_

In [ ]:
most_relevant_features= list( dict((k, v) for k, v in model.get_booster().get_fscore().items() if v >= 10).keys())
print(most_relevant_features)

In [ ]:
train_x = X_train[most_relevant_features]

In [ ]:
best_xgb_model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.3,
                 max_depth=6,
                 min_child_weight=1,
                 n_estimators=100,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
best_xgb_model.fit(train_x,y_train)

In [ ]:
dev_in_x=X_dev_in[most_relevant_features]

In [ ]:
pred = best_xgb_model.predict(dev_in_x)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, train_x, y_train, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_dev_in, pred)